In [1]:
import pydiscamb

# Load structure

Structures can be loaded using cctbx, from pdb-files or cif-files. Here, only pdb is shown, but loading cif is similar.
Additionally, Fobs can be read from a mtz-file, or from a cif. Only mtz is shown.

In [2]:
import iotbx.pdb
import mmtbx.model
from iotbx import reflection_file_reader

pdb_filename = "4znn.pdb"
pdb_inp = iotbx.pdb.input(file_name=pdb_filename)
model = mmtbx.model.manager(model_input=pdb_inp)
xrs = model.get_xray_structure()

# Scattering table is read from the structure
xrs.scattering_type_registry(table="electron")

# Load Fobs
mtz_filename = "4znn.mtz"
miller_arrays = reflection_file_reader.any_reflection_file(
    file_name=mtz_filename
).as_miller_arrays()
for ma in miller_arrays:
    if ma.info().label_string() == "FP,SIGFP":
        f_obs = ma

# Structure factor calculations

In [3]:
## The simplest usage: calculate structure factors in one call
fcalc = pydiscamb.calculate_structure_factors_IAM(xrs, d_min=2)

# TAAM is also available in this way:
fcalc = pydiscamb.calculate_structure_factors_TAAM(xrs, d_min=2)

Atomic form factors for electron scattering calculated with
The Hansen - Coppens model parameterized with MATTS databank.



`fcalc` is a `list` of `complex`, and does not retain information about the corresponding hkls.
These can be re-constructed with cctbx:

In [4]:
xrs.structure_factors(d_min=2).miller_set().indices()

This is convoluted. For more control, use `DiscambWrapper`:

In [5]:
wrapper = pydiscamb.DiscambWrapper(xrs)

# Either set d_min:
wrapper.set_d_min(2.0)
fcalc = wrapper.f_calc()

# Or set indices manually
wrapper.set_indices([(0, 1, 0), (2, 3, 1)])
fcalc = wrapper.f_calc()

# Also works with miller.indices()
wrapper.set_indices(f_obs.indices())
fcalc = wrapper.f_calc()


# TAAM
DiSCaMB's TAAM functionality is available:

In [6]:
taam_wrapper = pydiscamb.DiscambWrapper(xrs, method=pydiscamb.FCalcMethod.TAAM)
fcalc = taam_wrapper.f_calc(2.0) # Can also set d_min like this, in the same call

Atomic form factors for electron scattering calculated with
The Hansen - Coppens model parameterized with MATTS databank.



More control for e.g. logging can be achieved with `DiscambWrapper.from_TAAM_parameters`:

In [7]:
wrapper = pydiscamb.DiscambWrapper.from_TAAM_parameters(
    structure=xrs,
    convert_to_electron_scattering=False,
    bank_filepath=pydiscamb.taam_parameters.get_default_databank(),
    assignment_log_filepath="atom_type_assignment.log",
    parameter_log_filepath="multipolar_parameters.log",
    multipolar_cif_output_filepath="structure.cif",
    unit_cell_charge=0,
    perform_parameter_scaling_from_unit_cell_charge=True,
)
fcalc = wrapper.f_calc(2.0)

# The parameters are described in the function documentation
help(pydiscamb.DiscambWrapper.from_TAAM_parameters)

Help on built-in function from_TAAM_parameters in module pydiscamb._wrapper:

from_TAAM_parameters(...) method of builtins.PyCapsule instance
    from_TAAM_parameters(structure: object, convert_to_electron_scattering: bool, bank_filepath: str, assignment_log_filepath: str, parameter_log_filepath: str, multipolar_cif_output_filepath: str, unit_cell_charge: float, perform_parameter_scaling_from_unit_cell_charge: bool) -> pydiscamb._wrapper.DiscambWrapper
    
    
    Initialize a wrapper object with specified TAAM parameters. 
    
    Parameters
    ----------
    structure
        xray-structure to use
    convert_to_electron_scattering
        Whether to convert bank entries to electron using Mott-Bethe
    bank_filepath
        Path to databank of multipolar parameteres, e.g MATTS.
    assignment_log_filepath
        Path to output log file for atom assignment
    parameter_log_filepath
        Path to output log file for scattering parameters
    multipolar_cif_output_filepath
    

Atomic form factors for X-ray scattering calculated with
The Hansen - Coppens model parameterized with MATTS databank.



In [8]:
# List available banks in the installation
banks = pydiscamb.get_TAAM_databanks()
banks

['/home/viljar/anaconda3/envs/pyDiSCaMB_dev/lib/python3.8/site-packages/pydiscamb/data/empty_TAAM_databank.txt',
 '/home/viljar/anaconda3/envs/pyDiSCaMB_dev/lib/python3.8/site-packages/pydiscamb/data/MATTS2021databank.txt']

# Structure factor gradients
For refinement purposes, DiSCaMB's gradient calculations are exposed

In [9]:
# Set up wrapper
wrapper = pydiscamb.DiscambWrapper(xrs)
wrapper.set_indices(f_obs.indices())

# Get derivatives of each Fcalc with respect to each site, adp, occupancy ect.
f_calc_derivatives = wrapper.d_f_calc_d_params()

# Output is list of derivative objects, indices correspond to indices in f_obs.indices()
single_f_calc_derivatives = f_calc_derivatives[0]
# Corresponding hkl is therefore f_obs.indices()[0]

# Result object contains lists of parameter derivatives.
# Indices correspond to scattereres in the structure
site_derivatives = single_f_calc_derivatives.site_derivatives
first_atom_xyz = site_derivatives[0]
# Corresponding scatterer ("first atom") is therefore xrs.scatterers()[0]

We can also calculate derivatives with respect to a target function, provided the derivatives of the target with respect to each Fcalc

In [10]:
# First, get target
from mmtbx import f_model
model = f_model.manager(f_obs=f_obs, xray_structure=xrs)
target = model.target_functor()(compute_gradients=True)
d_target_d_fcalc = target.d_target_d_f_calc_work()

# Calculate gradients of target
gradients = wrapper.d_target_d_params(list(d_target_d_fcalc.data()))

# Output is now similar to the earlier output for a single hkl
# However, the indices now correspond directly to scatterers
single_atom_gradients = gradients[0]
# Corresponds to xrs.scatterers()[0]
xyz = single_atom_gradients.site_derivatives
adps = single_atom_gradients.adp_derivatives
# Can be 1 or 6 elements in the adp list, depending on xrs (uiso/uaniso)